<a href="https://colab.research.google.com/github/fcoliveira-utfpr/IDF_equation/blob/main/baixar_dados_xavier_por_zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando API do GEE e bibliotecas python

In [2]:
#!pip install google-api-python-client    -q
!pip install earthengine-api     -q

In [8]:
#importante modulos do GEE e autorização
import ee
# Trigger the authentication flow.
ee.Authenticate()
#intialize the google earth api
ee.Initialize(project='fcoliveira')
#importantando bibliotecas
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
import datetime
from datetime import datetime
from datetime import timedelta
import ee

##Função para baixar precipitação

In [9]:
# Definições da coleção e parâmetros
colecao_path = "projects/sat-io/open-datasets/BR-DWGD/PR"
banda = "b1"
offset = 225
escala = 0.006866665

# Função para baixar e organizar os dados
def processar_colecao(colecao_path, banda, offset, escala, ano, ponto, municipio):
    data_inicial = ee.Date.fromYMD(ano, 1, 1)
    data_final = ee.Date.fromYMD(ano, 12, 31)

    colecao = (
        ee.ImageCollection(colecao_path)
        .filterDate(data_inicial, data_final)
        .select([banda])
        .filterBounds(ponto)
    )

    dados = colecao.getRegion(ponto, 10, "epsg:4674").getInfo()

    if len(dados) < 2:
        return None  # Retorna None se não houver dados

    df = pd.DataFrame(dados[1:], columns=dados[0])  # Remove cabeçalho duplicado
    df = df[['time', banda]]

    df['time'] = pd.to_datetime(df['time'], unit='ms').dt.strftime('%Y-%m-%d')
    df[banda] = df[banda].astype(float) * escala + offset
    df[banda] = df[banda].apply(lambda x: max(x, 0) if pd.notna(x) else None)

    # Renomeia a coluna da chuva para incluir o nome do município
    df.rename(columns={banda: f"Chuva_{municipio}"}, inplace=True)

    return df

#Importando df com localização das ZH 7801

In [10]:
import pandas as pd
url = "https://raw.githubusercontent.com/fcoliveira-utfpr/IDF_equation/refs/heads/main/clima_zh_7801.csv"
df = pd.read_csv(url)
cidade = df['Municipio']
df = df.drop(columns=['Municipio'])
df = df.replace({',': '.'}, regex=True)
df = df.apply(lambda x: pd.to_numeric(x, errors = 'coerce'), axis=1)
df['Municipio'] = cidade
df_municipios = df.copy()
df_municipios

,ZH,Latitude,Longitude,Municipio
0,7801.0,-24.644085,-53.143762,Anahy
1,7801.0,-24.344029,-51.815311,Arapuã
2,7801.0,-24.377298,-51.540724,Ariranha do Ivaí
3,7801.0,-24.256862,-52.751176,Boa Esperança
4,7801.0,-25.431969,-53.421831,Boa Vista da Aparecida
5,7801.0,-24.616609,-52.788321,Campina da Lagoa
6,7801.0,-24.102494,-52.380703,Campo Mourão
7,7801.0,-25.610530,-53.760601,Capanema
8,7801.0,-25.469578,-53.593287,Capitão Leônidas Marques
9,7801.0,-25.246277,-53.761786,Céu Azul


##Aplicando a função e organizando dataframe

In [12]:
# Lista de anos
anos = list(range(1991, 2021+1))

# Criar DataFrame base
df_final = None

# Iterar sobre cada ano e município, processando um de cada vez
for ano in anos:
    df_ano = None  # DataFrame temporário para armazenar os dados do ano

    for _, row in df_municipios.iterrows():
        ponto = ee.Geometry.Point([row["Longitude"], row["Latitude"]])

        df_resultado = processar_colecao(colecao_path, banda, offset, escala, ano, ponto, row["Municipio"])

        if df_resultado is not None:
            if df_ano is None:
                df_ano = df_resultado
            else:
                df_ano = pd.merge(df_ano, df_resultado, on="time", how="outer")  # Une as colunas por data

    if df_ano is not None:
        df_ano["Ano"] = ano  # Adiciona a informação do ano
        if df_final is None:
            df_final = df_ano
        else:
            df_final = pd.concat([df_final, df_ano], ignore_index=True)

# Exibir resultado
df_final


,time,Chuva_Anahy,Chuva_Arapuã,Chuva_Ariranha do Ivaí,Chuva_Boa Esperança,Chuva_Boa Vista da Aparecida,Chuva_Campina da Lagoa,Chuva_Campo Mourão,Chuva_Capanema,Chuva_Capitão Leônidas Marques,...,Chuva_Santa Lúcia,Chuva_Santa Terezinha de Itaipu,Chuva_São José das Palmeiras,Chuva_São Miguel do Iguaçu,Chuva_São Pedro do Iguaçu,Chuva_Serranópolis do Iguaçu,Chuva_Tupãssi,Chuva_Ubiratã,Chuva_Vera Cruz do Oeste,Ano
0,1991-01-01,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1991
1,1991-01-02,0.000000,0.000000,0.000000,7.573919,0.00000,0.000000,2.952654,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1991
2,1991-01-03,9.372986,10.018452,10.588385,19.858383,5.01952,20.160516,8.562719,12.758252,9.297452,...,9.242519,3.138054,3.096854,13.053518,1.538121,26.093315,3.268520,7.113853,14.873184,1991
3,1991-01-04,1.036854,7.312986,3.302854,2.300321,4.87532,0.205988,2.574987,1.799054,6.022053,...,6.804853,0.000000,4.943987,0.295255,0.329588,2.787854,1.627388,2.059987,0.460054,1991
4,1991-01-05,0.000000,0.000000,0.000000,0.000000,0.00000,3.989520,0.000000,0.192255,0.000000,...,0.000000,0.000000,7.800519,0.000000,6.722453,0.000000,0.000000,0.000000,7.807386,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11287,2021-12-26,0.000000,0.720988,0.590521,1.641121,0.00000,0.000000,1.702921,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2021
11288,2021-12-27,0.679788,1.036854,1.023121,7.258053,0.00000,5.623787,4.332854,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.181054,0.000000,2021
11289,2021-12-28,0.000000,0.254055,5.143120,0.000000,0.00000,0.034321,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041188,0.000000,2021
11290,2021-12-29,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.281521,0.000000,0.000000,2021


In [15]:
#para baixar para excel os dados tire o # da última linha
from google.colab import files
nome_cidade = "chuva_max_7801"
base_dados = "xavier"
df_final.to_csv(f'{nome_cidade}_{base_dados}.csv')
files.download(f'{nome_cidade}_{base_dados}.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>